In [1]:
# Import required modules
from selenium import webdriver
import pandas as pd
import time
import numpy as np

In [2]:
# Scrape review by a brand
def scrape_review_by_brand(url):
    """url = profile page link of a brand"""
    
    # Initialize drivers
    driver = webdriver.Chrome("/home/faysal/Documents/utilities/chromedriver")
    driver.get(url)
    driver.maximize_window()
    time.sleep(4)
    
    # Check if the "View more" button exists
    try:
        view_more = driver.find_element_by_css_selector(".btn.btn-link.font-weight-bold")
    except:
        pass
    
    # Click until "View more" button disappears
    while True:
        try:
            view_more.click()
            time.sleep(2)
        except:
            print("No more pages to load!")
            break
    
    # Empty list of variables to be scraped
    course_title = []
    reviewed_by = []
    review = []
    date = []
    rating = []
    
    # Main container for every review
    main_container = driver.find_elements_by_css_selector(".review-content")
    
    # Loop through every main container
    for cont in main_container:
        
        # Scrape course title
        try:
            course_title.append(cont.find_element_by_css_selector(".heading.mt-2.ml-1.mr-1").text.strip())
        except:
            course_title.append("na")
        
        # Scrape reviewer
        try:
            reviewed_by.append(cont.find_element_by_css_selector(".mt-1.text-muted.subtext.d-md-flex.ml-1.mr-1").text.strip())
        except:
            reviewed_by.append("na")
        
        # Scrape the review
        try:
            review.append(cont.find_element_by_css_selector(".mt-1.ml-1.mr-1").text.strip())
        except:
            review.append("na")
        
        # Scrape data
        try:
            date.append(cont.find_element_by_css_selector(".text-muted.subtext").text.strip())
        except:
            date.append("na")
        
        # Scrape rating
        try:
            rating.append(cont.find_elements_by_css_selector("div")[1].get_attribute("alt").split("/")[0].strip())
        except:
            rating.append("na")
            
    # Create a temporary df
    temp_df = pd.DataFrame({
        "course_title":course_title,
        "reviewed_by":reviewed_by,
        "review":review,
        "date":date,
        "rating":rating
    })
    
    # Extract the actual reviewer
    temp_df["reviewer"] = temp_df.reviewed_by.str.split("|").str[0].str.strip().str.replace("Reviewed by", "")
    
    # Rows with purchase date
    has_purchase_date = temp_df[temp_df.reviewed_by.str.contains("Purchased", na=False, case=False)]
    has_purchase_date["purchased_on"] = has_purchase_date.reviewed_by.str.split("|").str[-1].str.strip().str.replace("Purchased", "")
    
    # Rows without purchase date
    has_not_purchase_date = temp_df[~temp_df.reviewed_by.str.contains("Purchased", na=False, case=False)]
    has_not_purchase_date["purchased_on"] = np.nan
    
    # Merge
    merged = pd.concat([has_purchase_date, has_not_purchase_date]).reset_index(drop=True)
    return merged.drop("reviewed_by", axis=1)

In [3]:
# Apply the function for a provider
df_review = scrape_review_by_brand("https://www.reed.co.uk/courses/profile/excel-with-business")
df_review.head(10)

No more pages to load!


<ipython-input-2-44924c285ca2>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  has_purchase_date["purchased_on"] = has_purchase_date.reviewed_by.str.split("|").str[-1].str.strip().str.replace("Purchased", "")
<ipython-input-2-44924c285ca2>:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  has_not_purchase_date["purchased_on"] = np.nan


,course_title,review,date,rating,reviewer,purchased_on
0,The Ultimate Microsoft Excel for Business Boot...,Straightforward and logically laid out so far....,12 Sep 2021,4.0,Julie Bridger,a month ago
1,Data Science Online Course,,24 Aug 2021,5.0,A Chowdhury,a year ago
2,Critical Thinking & Problem Solving Skills Course,The course video quality and the sound quality...,24 Aug 2021,5.0,A Chowdhury,3 years ago
3,Mega Microsoft Office 365 Bundle,This course version is not UK which is where I...,6 Aug 2021,1.0,Joann Watt,2 months ago
4,Microsoft Office & Windows Special Bundle,Have to say I was a little disappointed as whe...,4 Jul 2021,2.3,Paula Cooper,4 months ago
5,The Ultimate Microsoft Excel for Business Boot...,Good content for an advanced course on Excel. ...,15 May 2021,5.0,Julian,5 months ago
6,The Ultimate Microsoft Excel for Business Boot...,"A very thorough and informative course, provid...",29 Mar 2021,4.6,Angela Metcalfe,6 months ago
7,The Ultimate Microsoft Office Bundle Lifetime,Mainly text. Better to buy book and use YouTube,20 Mar 2021,1.6,Shoaib Qureshi,7 months ago
8,The Ultimate Microsoft Excel for Business Boot...,"I am so far very impressed with this course, l...",24 Feb 2021,5.0,Charlotte,7 months ago
9,The Ultimate Microsoft Office Bundle Lifetime,It felt like a long time between purchasing an...,21 Feb 2021,4.6,Angela Robertson,8 months ago
